Extração de Dados via API da Binance


    ## BTC - Mercado Futuro
    

In [6]:
##Importar Bibliotecas
import requests
import pandas as pd
import datetime
import polars as pl

In [7]:
# Definir o par de negociação e o tipo de contrato
pair = "BTCUSDT"
contract_type = "PERPETUAL"

# Definir o intervalo de tempo e o limite de registros
interval = "1d"
limit = 1500

# Construir a URL da API da Binance
base_url = "https://fapi.binance.com"
endpoint = f"/fapi/v1/klines?symbol={pair}&contractType={contract_type}&interval={interval}&limit={limit}"
url = base_url + endpoint

# Fazer a requisição à API da Binance
response = requests.get(url)

# Converter a resposta em uma lista de listas
data = response.json()

# Definir os nomes das colunas
columns = ["open_time", "open", "high", "low", "close", "volume", "close_time", "base_volume", "trades", "quote_volume", "asset_volume", "ignore"]

# Criar um DataFrame do pandas com os dados
df_btc = pd.DataFrame(data, columns=columns)
df_btc.head()

,open_time,open,high,low,close,volume,close_time,base_volume,trades,quote_volume,asset_volume,ignore
0,1590624000000,9197.32,9621.65,9110.97,9569.59,327412.374,1590710399999,3062477084.82549,801469,172464.347,1613575194.62399,0
1,1590710400000,9569.58,9598.00,9317.42,9422.26,238691.299,1590796799999,2255365978.23178,627717,118443.514,1119335783.23148,0
2,1590796800000,9422.11,9753.00,9326.01,9695.00,255234.058,1590883199999,2431044087.19118,691896,132940.629,1266665559.45577,0
3,1590883200000,9694.38,9698.00,9371.76,9446.43,219829.356,1590969599999,2096318341.55532,676486,105922.349,1010305260.93969,0
4,1590969600000,9446.49,10497.25,9420.85,10232.48,333740.457,1591055999999,3267274407.65878,922467,177298.140,1736224692.18492,0


        ## BTC - Fear & Greed

In [8]:
def get_fear_and_greed_index_history():
    url = 'https://api.alternative.me/fng/?limit=0'  # URL da API
    response = requests.get(url)
    data = response.json()

    # Cria um DataFrame para armazenar o histórico
    history = pd.DataFrame(data['data'])
    history['timestamp'] = pd.to_datetime(history['timestamp'].astype(int), unit='s')
    history.set_index('timestamp', inplace=True)

    return history

# Use a função para obter o histórico
df_fear_greed = get_fear_and_greed_index_history()
df_fear_greed = df_fear_greed.reset_index()
df_fear_greed.head(3)

,timestamp,value,value_classification,time_until_update
0,2024-07-05,29,Fear,82928
1,2024-07-04,44,Fear,NaN
2,2024-07-03,50,Neutral,NaN


Tratamento de Campos

        ## BTC - Mercado Futuro

In [9]:
# Criar variavel nova para não perder o dado original
df_btc_type = df_btc

# Converter dados para o tipo float
df_btc_type = df_btc_type.astype({'open': 'float64'})
df_btc_type = df_btc_type.astype({'high': 'float64'})
df_btc_type = df_btc_type.astype({'low': 'float64'})
df_btc_type = df_btc_type.astype({'close': 'float64'})
df_btc_type = df_btc_type.astype({'volume': 'float64'})
df_btc_type = df_btc_type.astype({'base_volume': 'float64'})
df_btc_type = df_btc_type.astype({'quote_volume': 'float64'})
df_btc_type = df_btc_type.astype({'asset_volume': 'float64'})

# Converter colunas relacionadas a data para o fomato to_datetime

df_btc_type["open_time"] = pd.to_datetime(df_btc_type["open_time"], unit="ms")
df_btc_type["close_time"] = pd.to_datetime(df_btc_type["close_time"], unit="ms")

# Renomear colunas
df_btc_type = df_btc_type.rename(columns={'open_time': 'DATA'})

# Inserir Coluna com nome da moeda
df_btc_type['moeda']='BTCUSDT'

# Remover colunas descnecessária
df_btc_type = df_btc_type.drop(columns=['ignore'])

df_btc_type.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   DATA          1500 non-null   datetime64[ns]
 1   open          1500 non-null   float64       
 2   high          1500 non-null   float64       
 3   low           1500 non-null   float64       
 4   close         1500 non-null   float64       
 5   volume        1500 non-null   float64       
 6   close_time    1500 non-null   datetime64[ns]
 7   base_volume   1500 non-null   float64       
 8   trades        1500 non-null   int64         
 9   quote_volume  1500 non-null   float64       
 10  asset_volume  1500 non-null   float64       
 11  moeda         1500 non-null   object        
dtypes: datetime64[ns](2), float64(8), int64(1), object(1)
memory usage: 140.8+ KB


        ## BTC - Fear & Greed

In [10]:
    # Change column type to datetime64[ns] for column: 'timestamp'
df_fear_greed = df_fear_greed.astype({'timestamp': 'datetime64[ns]'})
    # Change column type to int64 for column: 'value'
df_fear_greed = df_fear_greed.astype({'value': 'int64'})
    # Change column type to category for column: 'value_classification'
df_fear_greed = df_fear_greed.astype({'value_classification': 'category'})
    # Drop column: 'time_until_update'
df_fear_greed = df_fear_greed.drop(columns=['time_until_update'])

df_fear_greed = df_fear_greed.rename(columns={'timestamp': 'DATA'})

df_fear_greed = df_fear_greed.rename(columns={'value': 'Fear_&_Greed'})
df_fear_greed = df_fear_greed.rename(columns={'value_classification': 'Fear_&_Greed_tag'})
df_fear_greed['moeda'] = "BTCUSDT"
df_fear_greed


,DATA,Fear_&_Greed,Fear_&_Greed_tag,moeda
0,2024-07-05,29,Fear,BTCUSDT
1,2024-07-04,44,Fear,BTCUSDT
2,2024-07-03,50,Neutral,BTCUSDT
3,2024-07-02,51,Neutral,BTCUSDT
4,2024-07-01,53,Neutral,BTCUSDT
...,...,...,...,...
2339,2018-02-05,11,Extreme Fear,BTCUSDT
2340,2018-02-04,24,Extreme Fear,BTCUSDT
2341,2018-02-03,40,Fear,BTCUSDT
2342,2018-02-02,15,Extreme Fear,BTCUSDT


- Cruzar todas as bases

        ## - Merge entre as bases

In [11]:
df_btc_FearGreed = pd.merge(df_btc_type,df_fear_greed, on=["moeda", "DATA"], how='left')

df_btc_FearGreed.info()
df_btc_FearGreed


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATA              1500 non-null   datetime64[ns]
 1   open              1500 non-null   float64       
 2   high              1500 non-null   float64       
 3   low               1500 non-null   float64       
 4   close             1500 non-null   float64       
 5   volume            1500 non-null   float64       
 6   close_time        1500 non-null   datetime64[ns]
 7   base_volume       1500 non-null   float64       
 8   trades            1500 non-null   int64         
 9   quote_volume      1500 non-null   float64       
 10  asset_volume      1500 non-null   float64       
 11  moeda             1500 non-null   object        
 12  Fear_&_Greed      1500 non-null   int64         
 13  Fear_&_Greed_tag  1500 non-null   category      
dtypes: category(1), datetime

,DATA,open,high,low,close,volume,close_time,base_volume,trades,quote_volume,asset_volume,moeda,Fear_&_Greed,Fear_&_Greed_tag
0,2020-05-28,9197.32,9621.65,9110.97,9569.59,327412.374,2020-05-28 23:59:59.999,3.062477e+09,801469,172464.347,1.613575e+09,BTCUSDT,41,Fear
1,2020-05-29,9569.58,9598.00,9317.42,9422.26,238691.299,2020-05-29 23:59:59.999,2.255366e+09,627717,118443.514,1.119336e+09,BTCUSDT,48,Neutral
2,2020-05-30,9422.11,9753.00,9326.01,9695.00,255234.058,2020-05-30 23:59:59.999,2.431044e+09,691896,132940.629,1.266666e+09,BTCUSDT,48,Neutral
3,2020-05-31,9694.38,9698.00,9371.76,9446.43,219829.356,2020-05-31 23:59:59.999,2.096318e+09,676486,105922.349,1.010305e+09,BTCUSDT,51,Neutral
4,2020-06-01,9446.49,10497.25,9420.85,10232.48,333740.457,2020-06-01 23:59:59.999,3.267274e+09,922467,177298.140,1.736225e+09,BTCUSDT,50,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2024-07-01,62766.10,63849.00,62502.20,62883.60,193937.938,2024-07-01 23:59:59.999,1.224976e+10,2206156,98856.630,6.244453e+09,BTCUSDT,53,Neutral
1496,2024-07-02,62883.70,63284.00,61800.00,62123.60,163101.243,2024-07-02 23:59:59.999,1.018761e+10,1764087,81181.232,5.071447e+09,BTCUSDT,51,Neutral
1497,2024-07-03,62123.50,62278.20,59299.30,60208.00,295303.265,2024-07-03 23:59:59.999,1.786534e+10,2907975,141296.721,8.547986e+09,BTCUSDT,50,Neutral
1498,2024-07-04,60208.00,60500.00,56560.00,57045.90,445656.277,2024-07-04 23:59:59.999,2.587421e+10,4614925,212409.431,1.233124e+10,BTCUSDT,44,Fear


- Calcular Variações dos numeros entre os periodos

        ## BTC - Mercado Futuro - Preço

In [12]:
# Calcular a variação percentual por periodo de 7, 14, 21 e 28 dias

df_btc_type["var_1"] = df_btc_type.groupby(["moeda"])["close"].pct_change(periods=1)
df_btc_type["var_7"] = df_btc_type.groupby(["moeda"])["close"].pct_change(periods=7)
df_btc_type["var_14"] = df_btc_type.groupby(["moeda"])["close"].pct_change(periods=14)
df_btc_type["var_21"] = df_btc_type.groupby(["moeda"])["close"].pct_change(periods=21)
df_btc_type["var_28"] = df_btc_type.groupby(["moeda"])["close"].pct_change(periods=28)
df_btc_type.head()



# Capturar valor dos periodos anteriores
df_btc_type[["val_p1","val_p7","val_p14","val_p21","val_p28"]] = df_btc_type.groupby(["moeda"]) ["close"].shift(periods=[ 1, 7, 14, 21, 28])

In [13]:
# Calcular a variação percentual por periodo de 7, 14, 21 e 28 dias

df_btc_FearGreed["var_1"] = df_btc_FearGreed.groupby(["moeda"])["close"].pct_change(periods=1)
df_btc_FearGreed["var_7"] = df_btc_FearGreed.groupby(["moeda"])["close"].pct_change(periods=7)
df_btc_FearGreed["var_14"] = df_btc_FearGreed.groupby(["moeda"])["close"].pct_change(periods=14)
df_btc_FearGreed["var_21"] = df_btc_FearGreed.groupby(["moeda"])["close"].pct_change(periods=21)
df_btc_FearGreed["var_28"] = df_btc_FearGreed.groupby(["moeda"])["close"].pct_change(periods=28)
df_btc_FearGreed.head()



# Capturar valor dos periodos anteriores
df_btc_FearGreed[["val_p1","val_p7","val_p14","val_p21","val_p28"]] = df_btc_FearGreed.groupby(["moeda"]) ["close"].shift(periods=[ 1, 7, 14, 21, 28])

        ## BTC - Mercado Futuro - Volume

In [14]:

# Calcular a variação percentual por periodo de 7, 14, 21 e 28 dias

df_btc_FearGreed["varVol_1"] = df_btc_FearGreed.groupby(["moeda"])["base_volume"].pct_change(periods=1)
df_btc_FearGreed["varVol_7"] = df_btc_FearGreed.groupby(["moeda"])["base_volume"].pct_change(periods=7)
df_btc_FearGreed["varVol_14"] = df_btc_FearGreed.groupby(["moeda"])["base_volume"].pct_change(periods=14)
df_btc_FearGreed["varVol_21"] = df_btc_FearGreed.groupby(["moeda"])["base_volume"].pct_change(periods=21)
df_btc_FearGreed["varVol_28"] = df_btc_FearGreed.groupby(["moeda"])["base_volume"].pct_change(periods=28)

# Capturar valor dos periodos anteriores
df_btc_FearGreed[["valVol_p1","valVol_p7","valVol_p14","valVol_p21","valVol_p28"]] = df_btc_FearGreed.groupby(["moeda"]) ["base_volume"].shift(periods=[ 1, 7, 14, 21, 28])





        ## BTC - Mercado Futuro - Fear & Greed

In [15]:

# Calcular a variação percentual por periodo de 7, 14, 21 e 28 dias

df_btc_FearGreed["varFeG_1"] = df_btc_FearGreed.groupby(["moeda"])["Fear_&_Greed"].pct_change(periods=1)
df_btc_FearGreed["varFeG_7"] = df_btc_FearGreed.groupby(["moeda"])["Fear_&_Greed"].pct_change(periods=7)
df_btc_FearGreed["varFeG_14"] = df_btc_FearGreed.groupby(["moeda"])["Fear_&_Greed"].pct_change(periods=14)
df_btc_FearGreed["varFeG_21"] = df_btc_FearGreed.groupby(["moeda"])["Fear_&_Greed"].pct_change(periods=21)
df_btc_FearGreed["varFeG_28"] = df_btc_FearGreed.groupby(["moeda"])["Fear_&_Greed"].pct_change(periods=28)

# Capturar valor dos periodos anteriores
df_btc_FearGreed[["valFeG_p1","valFeG_p7","valFeG_p14","valFeG_p21","valFeG_p28"]] = df_btc_FearGreed.groupby(["moeda"]) ["Fear_&_Greed"].shift(periods=[ 1, 7, 14, 21, 28])



Criação de Indicadores

        ## Media Movel de 200 dias
    

In [16]:
# Calcular a média móvel de 200 dias do preço de fechamento do bitcoin
df_btc_FearGreed["ma200"] = df_btc_FearGreed["close"].rolling(200).mean()

In [17]:
# Calcular a média móvel de 200 dias do preço de fechamento do bitcoin
df_btc_type["ma200"] = df_btc_type["close"].rolling(200).mean()

        ## Mayer - "Quando a relação da média movel de 200 dias é positiva ou negativa"

In [18]:

# Calcular o Múltiplo de Mayer
df_btc_FearGreed["mayer"] = df_btc_FearGreed["close"] / df_btc_FearGreed["ma200"]




In [19]:

# Calcular o Múltiplo de Mayer
df_btc_type["mayer"] = df_btc_type["close"] / df_btc_type["ma200"]


        ## Tag para relacionar o "Mayer" que indica se o BTC esta sobrecomprado ou subvalorizado

In [20]:
# Criação de funcao para Categorizar o momento do Bitcoin com relação ao  indicador de Multiplo de mayer 200 

def tag_mayer(variavel):
    if variavel >1 :
        return 'SobreComprado'
    elif variavel <1:
        return 'SubValorizado'
    else:
        return 'Nem Subvalorizado nem Sobrecomprado'
    
# Processar função de categorização do momento do Bitcoin com relação ao  indicador de Multiplo de mayer 200 
df_btc_type['tagmayer'] = df_btc_type['mayer'].apply(tag_mayer)

# Alterar tipo da coluna tagmayer para o formato "Category" 
df_btc_type = df_btc_type.astype({'tagmayer': 'category'})




In [12]:
# Criação de funcao para Categorizar o momento do Bitcoin com relação ao  indicador de Multiplo de mayer 200 

def tag_mayer(variavel):
    if variavel >1 :
        return 'SobreComprado'
    elif variavel <1:
        return 'SubValorizado'
    else:
        return 'Nem Subvalorizado nem Sobrecomprado'
    
# Processar função de categorização do momento do Bitcoin com relação ao  indicador de Multiplo de mayer 200 
df_btc_FearGreed['tagmayer'] = df_btc_FearGreed['mayer'].apply(tag_mayer)

# Alterar tipo da coluna tagmayer para o formato "Category" 
df_btc_FearGreed = df_btc_FearGreed.astype({'tagmayer': 'category'})
df_btc_FearGreed



,DATA,open,high,low,close,volume,close_time,base_volume,trades,quote_volume,...,varFeG_21,varFeG_28,valFeG_p1,valFeG_p7,valFeG_p14,valFeG_p21,valFeG_p28,ma200,mayer,tagmayer
0,2020-02-14,10240.02,10390.00,10123.37,10356.82,130975.746,2020-02-14 23:59:59.999,1.345002e+09,267110,67096.628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
1,2020-02-15,10356.27,10389.75,9752.30,9914.99,152146.127,2020-02-15 23:59:59.999,1.533601e+09,325934,70415.864,...,NaN,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
2,2020-02-16,9914.31,10060.45,9626.64,9921.86,141547.956,2020-02-16 23:59:59.999,1.398907e+09,321449,69642.561,...,NaN,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
3,2020-02-17,9921.87,9971.00,9461.89,9716.00,185338.179,2020-02-17 23:59:59.999,1.795990e+09,399493,89956.126,...,NaN,NaN,59.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
4,2020-02-18,9716.00,10270.00,9598.00,10174.32,166119.369,2020-02-18 23:59:59.999,1.642321e+09,398826,84019.968,...,NaN,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2024-03-19,67660.00,68178.60,61582.80,61974.10,700466.795,2024-03-19 23:59:59.999,4.502241e+10,8883840,341387.210,...,0.000000,0.097222,77.0,81.0,90.0,79.0,72.0,40792.1415,1.519266,SobreComprado
1496,2024-03-20,61974.10,68140.00,60800.00,67876.40,637362.465,2024-03-20 23:59:59.999,4.069112e+10,8081289,322745.423,...,-0.097561,-0.051282,79.0,81.0,75.0,82.0,78.0,41002.2445,1.655431,SobreComprado
1497,2024-03-21,67876.40,68293.90,64550.10,65539.90,347218.385,2024-03-21 23:59:59.999,2.311429e+10,5314298,170701.466,...,-0.025000,0.054054,74.0,88.0,82.0,80.0,74.0,41200.1335,1.590769,SobreComprado
1498,2024-03-22,65539.90,66691.90,62294.00,63826.00,401149.333,2024-03-22 23:59:59.999,2.574280e+10,5610913,193242.858,...,-0.062500,-0.013158,78.0,83.0,81.0,80.0,76.0,41390.1935,1.542056,SobreComprado


- Export do Resultado Adicional

In [21]:

# Ordenar por ultima data
df_btc_FearGreed = df_btc_FearGreed.sort_values('DATA', ascending=False)

# Seleciona as linhas do índice 1 ao 30 (desconsidera o índice 0 que é o item mais recente)
Registros_10_head = df_btc_FearGreed.iloc[1:10]  

Registros_10_head



,DATA,open,high,low,close,volume,close_time,base_volume,trades,quote_volume,...,varFeG_14,varFeG_21,varFeG_28,valFeG_p1,valFeG_p7,valFeG_p14,valFeG_p21,valFeG_p28,ma200,mayer
1498,2024-07-04,60208.0,60500.0,56560.0,57045.9,445656.277,2024-07-04 23:59:59.999,2.587421e+10,4614925,212409.431,...,-0.266667,-0.371429,-0.435897,50.0,40.0,60.0,70.0,78.0,58491.1765,0.975291
1497,2024-07-03,62123.5,62278.2,59299.3,60208.0,295303.265,2024-07-03 23:59:59.999,1.786534e+10,2907975,141296.721,...,-0.218750,-0.305556,-0.333333,51.0,46.0,64.0,72.0,75.0,58412.9295,1.030731
1496,2024-07-02,62883.7,63284.0,61800.0,62123.6,163101.243,2024-07-02 23:59:59.999,1.018761e+10,1764087,81181.232,...,-0.310811,-0.310811,-0.301370,53.0,30.0,74.0,74.0,73.0,58323.3265,1.065159
1495,2024-07-01,62766.1,63849.0,62502.2,62883.6,193937.938,2024-07-01 23:59:59.999,1.224976e+10,2206156,98856.630,...,-0.253521,-0.263889,-0.273973,47.0,51.0,71.0,72.0,73.0,58222.5485,1.080056
1494,2024-06-30,60982.5,63058.4,60705.0,62766.0,147777.915,2024-06-30 23:59:59.999,9.131797e+09,1720176,76847.014,...,-0.338028,-0.373333,-0.356164,30.0,55.0,71.0,75.0,73.0,58123.2925,1.079877
1493,2024-06-29,60429.8,61232.0,60400.1,60982.4,79680.289,2024-06-29 23:59:59.999,4.854467e+09,1008124,39587.752,...,-0.594595,-0.583333,-0.583333,47.0,53.0,74.0,72.0,72.0,58023.8375,1.050989
1492,2024-06-28,61682.1,62204.0,60038.0,60429.8,195508.125,2024-06-28 23:59:59.999,1.196316e+10,2261011,96784.819,...,-0.364865,-0.389610,-0.356164,40.0,63.0,74.0,77.0,73.0,57926.5005,1.043215
1491,2024-06-27,60860.1,62379.7,60605.0,61682.0,160172.482,2024-06-27 23:59:59.999,9.845346e+09,1943987,80692.923,...,-0.428571,-0.487179,-0.452055,46.0,60.0,70.0,78.0,73.0,57830.7065,1.066596
1490,2024-06-26,61777.9,62465.1,60696.0,60860.1,170601.180,2024-06-26 23:59:59.999,1.047422e+10,2072190,85546.810,...,-0.361111,-0.386667,-0.361111,30.0,64.0,72.0,75.0,72.0,57741.3465,1.054012


In [22]:
Registros_10_head = Registros_10_head.reset_index()


In [23]:
Registros_10_head = Registros_10_head.drop(columns=['index'])
Registros_10_head

,DATA,open,high,low,close,volume,close_time,base_volume,trades,quote_volume,...,varFeG_14,varFeG_21,varFeG_28,valFeG_p1,valFeG_p7,valFeG_p14,valFeG_p21,valFeG_p28,ma200,mayer
0,2024-07-04,60208.0,60500.0,56560.0,57045.9,445656.277,2024-07-04 23:59:59.999,2.587421e+10,4614925,212409.431,...,-0.266667,-0.371429,-0.435897,50.0,40.0,60.0,70.0,78.0,58491.1765,0.975291
1,2024-07-03,62123.5,62278.2,59299.3,60208.0,295303.265,2024-07-03 23:59:59.999,1.786534e+10,2907975,141296.721,...,-0.218750,-0.305556,-0.333333,51.0,46.0,64.0,72.0,75.0,58412.9295,1.030731
2,2024-07-02,62883.7,63284.0,61800.0,62123.6,163101.243,2024-07-02 23:59:59.999,1.018761e+10,1764087,81181.232,...,-0.310811,-0.310811,-0.301370,53.0,30.0,74.0,74.0,73.0,58323.3265,1.065159
3,2024-07-01,62766.1,63849.0,62502.2,62883.6,193937.938,2024-07-01 23:59:59.999,1.224976e+10,2206156,98856.630,...,-0.253521,-0.263889,-0.273973,47.0,51.0,71.0,72.0,73.0,58222.5485,1.080056
4,2024-06-30,60982.5,63058.4,60705.0,62766.0,147777.915,2024-06-30 23:59:59.999,9.131797e+09,1720176,76847.014,...,-0.338028,-0.373333,-0.356164,30.0,55.0,71.0,75.0,73.0,58123.2925,1.079877
5,2024-06-29,60429.8,61232.0,60400.1,60982.4,79680.289,2024-06-29 23:59:59.999,4.854467e+09,1008124,39587.752,...,-0.594595,-0.583333,-0.583333,47.0,53.0,74.0,72.0,72.0,58023.8375,1.050989
6,2024-06-28,61682.1,62204.0,60038.0,60429.8,195508.125,2024-06-28 23:59:59.999,1.196316e+10,2261011,96784.819,...,-0.364865,-0.389610,-0.356164,40.0,63.0,74.0,77.0,73.0,57926.5005,1.043215
7,2024-06-27,60860.1,62379.7,60605.0,61682.0,160172.482,2024-06-27 23:59:59.999,9.845346e+09,1943987,80692.923,...,-0.428571,-0.487179,-0.452055,46.0,60.0,70.0,78.0,73.0,57830.7065,1.066596
8,2024-06-26,61777.9,62465.1,60696.0,60860.1,170601.180,2024-06-26 23:59:59.999,1.047422e+10,2072190,85546.810,...,-0.361111,-0.386667,-0.361111,30.0,64.0,72.0,75.0,72.0,57741.3465,1.054012


In [29]:
base_Atual = pl.read_parquet(r'..\03_output\BTC_OFICIAL.parquet')


#converte dataset em pandas 
base_Atual = base_Atual.to_pandas()

# Ordenar por ultima data
base_Atual = base_Atual.sort_values('DATA', ascending=False)

# Remove a última linha usando drop()
base_Atual = base_Atual.drop(base_Atual.index[0])

# Remove duplicadas
base_Atual = base_Atual.drop_duplicates()

print(base_Atual['DATA'].value_counts())

base_Atual



DATA
2020-02-02    1
2024-03-21    1
2024-03-20    1
2024-03-19    1
2024-03-18    1
             ..
2024-03-11    1
2024-03-12    1
2024-03-13    1
2024-03-14    1
2024-03-15    1
Name: count, Length: 1510, dtype: int64


,DATA,open,high,low,close,volume,close_time,base_volume,trades,quote_volume,...,varFeG_21,varFeG_28,valFeG_p1,valFeG_p7,valFeG_p14,valFeG_p21,valFeG_p28,ma200,mayer,tagmayer
1,2024-03-21,67876.40,68293.90,64550.10,65539.90,347218.385,2024-03-21 23:59:59.999,2.311429e+10,5314298,170701.466,...,-0.025000,0.054054,74.0,88.0,82.0,80.0,74.0,41200.1335,1.590769,SobreComprado
2,2024-03-20,61974.10,68140.00,60800.00,67876.40,637362.465,2024-03-20 23:59:59.999,4.069112e+10,8081289,322745.423,...,-0.097561,-0.051282,79.0,81.0,75.0,82.0,78.0,41002.2445,1.655431,SobreComprado
3,2024-03-19,67660.00,68178.60,61582.80,61974.10,700466.795,2024-03-19 23:59:59.999,4.502241e+10,8883840,341387.210,...,0.000000,0.097222,77.0,81.0,90.0,79.0,72.0,40792.1415,1.519266,SobreComprado
4,2024-03-18,68447.90,69044.40,66601.60,67660.00,317885.916,2024-03-18 23:59:59.999,2.152597e+10,4929189,157518.056,...,0.069444,0.026667,79.0,82.0,82.0,72.0,75.0,40611.2490,1.666041,SobreComprado
5,2024-03-17,65351.00,68972.40,64559.80,68447.90,314890.309,2024-03-17 23:59:59.999,2.102834e+10,4441202,157746.931,...,0.067568,0.097222,81.0,79.0,83.0,74.0,72.0,40402.5890,1.694146,SobreComprado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,2020-02-06,9630.00,9884.00,9550.00,9778.12,200556.367,2020-02-06 23:59:59.999,1.950865e+09,366953,98843.932,...,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
1507,2020-02-05,9198.32,9799.08,9179.88,9630.00,212311.697,2020-02-05 23:59:59.999,2.014987e+09,387332,111557.886,...,NaN,NaN,56.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
1508,2020-02-04,9300.63,9368.00,9105.00,9198.35,147843.902,2020-02-04 23:59:59.999,1.363688e+09,282503,71776.668,...,NaN,NaN,59.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
1509,2020-02-03,9340.49,9647.61,9250.00,9300.64,159049.385,2020-02-03 23:59:59.999,1.491077e+09,313233,79180.283,...,NaN,NaN,57.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado


In [30]:
base_Atual_Datas = pd.DataFrame(base_Atual['DATA'])
base_Atual_Datas

,DATA
1,2024-03-21
2,2024-03-20
3,2024-03-19
4,2024-03-18
5,2024-03-17
...,...
1506,2020-02-06
1507,2020-02-05
1508,2020-02-04
1509,2020-02-03


In [31]:
Registros_10_head_Datas = pd.DataFrame(Registros_10_head['DATA'])
Registros_10_head_Datas

,DATA
0,2024-07-04
1,2024-07-03
2,2024-07-02
3,2024-07-01
4,2024-06-30
5,2024-06-29
6,2024-06-28
7,2024-06-27
8,2024-06-26


In [32]:
Datas_para_concatenar = pd.merge(base_Atual_Datas,Registros_10_head_Datas,how='right', indicator=True)
Datas_para_concatenar = Datas_para_concatenar.query('_merge == "right_only"')


In [33]:
Datas_para_concatenar_selecionado = pd.merge(Datas_para_concatenar,Registros_10_head)
Datas_para_concatenar_selecionado

,DATA,_merge,open,high,low,close,volume,close_time,base_volume,trades,...,varFeG_14,varFeG_21,varFeG_28,valFeG_p1,valFeG_p7,valFeG_p14,valFeG_p21,valFeG_p28,ma200,mayer
0,2024-07-04,right_only,60208.0,60500.0,56560.0,57045.9,445656.277,2024-07-04 23:59:59.999,2.587421e+10,4614925,...,-0.266667,-0.371429,-0.435897,50.0,40.0,60.0,70.0,78.0,58491.1765,0.975291
1,2024-07-03,right_only,62123.5,62278.2,59299.3,60208.0,295303.265,2024-07-03 23:59:59.999,1.786534e+10,2907975,...,-0.218750,-0.305556,-0.333333,51.0,46.0,64.0,72.0,75.0,58412.9295,1.030731
2,2024-07-02,right_only,62883.7,63284.0,61800.0,62123.6,163101.243,2024-07-02 23:59:59.999,1.018761e+10,1764087,...,-0.310811,-0.310811,-0.301370,53.0,30.0,74.0,74.0,73.0,58323.3265,1.065159
3,2024-07-01,right_only,62766.1,63849.0,62502.2,62883.6,193937.938,2024-07-01 23:59:59.999,1.224976e+10,2206156,...,-0.253521,-0.263889,-0.273973,47.0,51.0,71.0,72.0,73.0,58222.5485,1.080056
4,2024-06-30,right_only,60982.5,63058.4,60705.0,62766.0,147777.915,2024-06-30 23:59:59.999,9.131797e+09,1720176,...,-0.338028,-0.373333,-0.356164,30.0,55.0,71.0,75.0,73.0,58123.2925,1.079877
5,2024-06-29,right_only,60429.8,61232.0,60400.1,60982.4,79680.289,2024-06-29 23:59:59.999,4.854467e+09,1008124,...,-0.594595,-0.583333,-0.583333,47.0,53.0,74.0,72.0,72.0,58023.8375,1.050989
6,2024-06-28,right_only,61682.1,62204.0,60038.0,60429.8,195508.125,2024-06-28 23:59:59.999,1.196316e+10,2261011,...,-0.364865,-0.389610,-0.356164,40.0,63.0,74.0,77.0,73.0,57926.5005,1.043215
7,2024-06-27,right_only,60860.1,62379.7,60605.0,61682.0,160172.482,2024-06-27 23:59:59.999,9.845346e+09,1943987,...,-0.428571,-0.487179,-0.452055,46.0,60.0,70.0,78.0,73.0,57830.7065,1.066596
8,2024-06-26,right_only,61777.9,62465.1,60696.0,60860.1,170601.180,2024-06-26 23:59:59.999,1.047422e+10,2072190,...,-0.361111,-0.386667,-0.361111,30.0,64.0,72.0,75.0,72.0,57741.3465,1.054012


In [34]:
# Concatenar resultados
dataset_concat = pd.concat([base_Atual,Datas_para_concatenar_selecionado]).reset_index()


In [22]:
dataset_concat = dataset_concat.drop(columns=['_merge'])
dataset_concat

,index,DATA,open,high,low,close,volume,close_time,base_volume,trades,...,varFeG_21,varFeG_28,valFeG_p1,valFeG_p7,valFeG_p14,valFeG_p21,valFeG_p28,ma200,mayer,tagmayer
0,1,2024-03-18,68447.90,69044.40,66601.60,67660.0,317885.916,2024-03-18 23:59:59.999,2.152597e+10,4929189,...,0.069444,0.026667,79.0,82.0,82.0,72.0,75.0,40611.2490,1.666041,SobreComprado
1,2,2024-03-17,65351.00,68972.40,64559.80,68447.9,314890.309,2024-03-17 23:59:59.999,2.102834e+10,4441202,...,0.067568,0.097222,81.0,79.0,83.0,74.0,72.0,40402.5890,1.694146,SobreComprado
2,3,2024-03-16,69552.10,70097.60,64750.00,65351.0,333426.129,2024-03-16 23:59:59.999,2.254172e+10,4728369,...,0.125000,0.065789,83.0,84.0,80.0,72.0,76.0,40196.8015,1.625776,SobreComprado
3,4,2024-03-15,71455.60,72500.00,65580.00,69552.2,705810.685,2024-03-15 23:59:59.999,4.827259e+10,9432499,...,0.092105,0.152778,88.0,81.0,80.0,76.0,72.0,40008.5470,1.738434,SobreComprado
4,5,2024-03-14,73127.60,73881.40,68600.00,71455.5,513901.237,2024-03-14 23:59:59.999,3.671656e+10,7086199,...,0.189189,0.222222,81.0,82.0,80.0,74.0,72.0,39791.3185,1.795756,SobreComprado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,1507,2020-02-02,9392.99,9489.92,9160.01,9338.2,148402.977,2020-02-02 23:59:59.999,1.391949e+09,285872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
1507,0,2024-03-22,65539.90,66691.90,62294.00,63826.0,401149.333,2024-03-22 23:59:59.999,2.574280e+10,5610913,...,-0.062500,-0.013158,78.0,83.0,81.0,80.0,76.0,41390.1935,1.542056,SobreComprado
1508,1,2024-03-21,67876.40,68293.90,64550.10,65539.9,347218.385,2024-03-21 23:59:59.999,2.311429e+10,5314298,...,-0.025000,0.054054,74.0,88.0,82.0,80.0,74.0,41200.1335,1.590769,SobreComprado
1509,2,2024-03-20,61974.10,68140.00,60800.00,67876.4,637362.465,2024-03-20 23:59:59.999,4.069112e+10,8081289,...,-0.097561,-0.051282,79.0,81.0,75.0,82.0,78.0,41002.2445,1.655431,SobreComprado


In [35]:
dataset_concat = dataset_concat.drop(columns=['index'])
dataset_concat

,DATA,open,high,low,close,volume,close_time,base_volume,trades,quote_volume,...,varFeG_28,valFeG_p1,valFeG_p7,valFeG_p14,valFeG_p21,valFeG_p28,ma200,mayer,tagmayer,_merge
0,2024-03-21,67876.4,68293.9,64550.1,65539.9,347218.385,2024-03-21 23:59:59.999,2.311429e+10,5314298,170701.466,...,0.054054,74.0,88.0,82.0,80.0,74.0,41200.1335,1.590769,SobreComprado,NaN
1,2024-03-20,61974.1,68140.0,60800.0,67876.4,637362.465,2024-03-20 23:59:59.999,4.069112e+10,8081289,322745.423,...,-0.051282,79.0,81.0,75.0,82.0,78.0,41002.2445,1.655431,SobreComprado,NaN
2,2024-03-19,67660.0,68178.6,61582.8,61974.1,700466.795,2024-03-19 23:59:59.999,4.502241e+10,8883840,341387.210,...,0.097222,77.0,81.0,90.0,79.0,72.0,40792.1415,1.519266,SobreComprado,NaN
3,2024-03-18,68447.9,69044.4,66601.6,67660.0,317885.916,2024-03-18 23:59:59.999,2.152597e+10,4929189,157518.056,...,0.026667,79.0,82.0,82.0,72.0,75.0,40611.2490,1.666041,SobreComprado,NaN
4,2024-03-17,65351.0,68972.4,64559.8,68447.9,314890.309,2024-03-17 23:59:59.999,2.102834e+10,4441202,157746.931,...,0.097222,81.0,79.0,83.0,74.0,72.0,40402.5890,1.694146,SobreComprado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,2024-06-30,60982.5,63058.4,60705.0,62766.0,147777.915,2024-06-30 23:59:59.999,9.131797e+09,1720176,76847.014,...,-0.356164,30.0,55.0,71.0,75.0,73.0,58123.2925,1.079877,NaN,right_only
1515,2024-06-29,60429.8,61232.0,60400.1,60982.4,79680.289,2024-06-29 23:59:59.999,4.854467e+09,1008124,39587.752,...,-0.583333,47.0,53.0,74.0,72.0,72.0,58023.8375,1.050989,NaN,right_only
1516,2024-06-28,61682.1,62204.0,60038.0,60429.8,195508.125,2024-06-28 23:59:59.999,1.196316e+10,2261011,96784.819,...,-0.356164,40.0,63.0,74.0,77.0,73.0,57926.5005,1.043215,NaN,right_only
1517,2024-06-27,60860.1,62379.7,60605.0,61682.0,160172.482,2024-06-27 23:59:59.999,9.845346e+09,1943987,80692.923,...,-0.452055,46.0,60.0,70.0,78.0,73.0,57830.7065,1.066596,NaN,right_only


In [24]:
# Ordenar por ultima data
dataset_concat = dataset_concat.sort_values('DATA', ascending=False)
dataset_concat

,DATA,open,high,low,close,volume,close_time,base_volume,trades,quote_volume,...,varFeG_21,varFeG_28,valFeG_p1,valFeG_p7,valFeG_p14,valFeG_p21,valFeG_p28,ma200,mayer,tagmayer
1507,2024-03-22,65539.90,66691.90,62294.00,63826.00,401149.333,2024-03-22 23:59:59.999,2.574280e+10,5610913,193242.858,...,-0.062500,-0.013158,78.0,83.0,81.0,80.0,76.0,41390.1935,1.542056,SobreComprado
1508,2024-03-21,67876.40,68293.90,64550.10,65539.90,347218.385,2024-03-21 23:59:59.999,2.311429e+10,5314298,170701.466,...,-0.025000,0.054054,74.0,88.0,82.0,80.0,74.0,41200.1335,1.590769,SobreComprado
1509,2024-03-20,61974.10,68140.00,60800.00,67876.40,637362.465,2024-03-20 23:59:59.999,4.069112e+10,8081289,322745.423,...,-0.097561,-0.051282,79.0,81.0,75.0,82.0,78.0,41002.2445,1.655431,SobreComprado
1510,2024-03-19,67660.00,68178.60,61582.80,61974.10,700466.795,2024-03-19 23:59:59.999,4.502241e+10,8883840,341387.210,...,0.000000,0.097222,77.0,81.0,90.0,79.0,72.0,40792.1415,1.519266,SobreComprado
0,2024-03-18,68447.90,69044.40,66601.60,67660.00,317885.916,2024-03-18 23:59:59.999,2.152597e+10,4929189,157518.056,...,0.069444,0.026667,79.0,82.0,82.0,72.0,75.0,40611.2490,1.666041,SobreComprado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,2020-02-06,9630.00,9884.00,9550.00,9778.12,200556.367,2020-02-06 23:59:59.999,1.950865e+09,366953,98843.932,...,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
1503,2020-02-05,9198.32,9799.08,9179.88,9630.00,212311.697,2020-02-05 23:59:59.999,2.014987e+09,387332,111557.886,...,NaN,NaN,56.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
1504,2020-02-04,9300.63,9368.00,9105.00,9198.35,147843.902,2020-02-04 23:59:59.999,1.363688e+09,282503,71776.668,...,NaN,NaN,59.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado
1505,2020-02-03,9340.49,9647.61,9250.00,9300.64,159049.385,2020-02-03 23:59:59.999,1.491077e+09,313233,79180.283,...,NaN,NaN,57.0,NaN,NaN,NaN,NaN,NaN,NaN,Nem Subvalorizado nem Sobrecomprado


In [36]:

dataset_concat = dataset_concat.drop_duplicates()
dataset_concat.info()
dataset_concat['DATA'].value_counts()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1519 entries, 0 to 1518
Data columns (total 48 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATA              1519 non-null   datetime64[ns]
 1   open              1519 non-null   float64       
 2   high              1519 non-null   float64       
 3   low               1519 non-null   float64       
 4   close             1519 non-null   float64       
 5   volume            1519 non-null   float64       
 6   close_time        1519 non-null   datetime64[ns]
 7   base_volume       1519 non-null   float64       
 8   trades            1519 non-null   int64         
 9   quote_volume      1519 non-null   float64       
 10  asset_volume      1519 non-null   float64       
 11  moeda             1519 non-null   object        
 12  Fear_&_Greed      1519 non-null   int64         
 13  Fear_&_Greed_tag  1519 non-null   object        
 14  var_1             1518 n

DATA
2024-06-26    1
2024-03-21    1
2024-03-20    1
2024-03-19    1
2024-03-18    1
             ..
2024-03-02    1
2024-03-03    1
2024-03-04    1
2024-03-05    1
2024-03-06    1
Name: count, Length: 1519, dtype: int64

In [37]:
#Converter DataFrame do pandas em DataFrame do polars e ganhar performance no export do output
dataset_concat_pl = pl.DataFrame(dataset_concat)
dataset_concat_pl

DATA,open,high,low,close,volume,close_time,base_volume,trades,quote_volume,asset_volume,moeda,Fear_&_Greed,Fear_&_Greed_tag,var_1,var_7,var_14,var_21,var_28,val_p1,val_p7,val_p14,val_p21,val_p28,varVol_1,varVol_7,varVol_14,varVol_21,varVol_28,valVol_p1,valVol_p7,valVol_p14,valVol_p21,valVol_p28,varFeG_1,varFeG_7,varFeG_14,varFeG_21,varFeG_28,valFeG_p1,valFeG_p7,valFeG_p14,valFeG_p21,valFeG_p28,ma200,mayer,tagmayer,_merge
datetime[ns],f64,f64,f64,f64,f64,datetime[ns],f64,i64,f64,f64,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,cat
2024-03-21 00:00:00,67876.4,68293.9,64550.1,65539.9,347218.385,2024-03-21 23:59:59.999,2.3114e10,5314298,170701.466,1.1368e10,"""BTCUSDT""",78,"""Extreme Greed""",-0.034423,-0.082787,-0.019891,0.070856,0.27716,67876.4,71455.5,66870.0,61203.3,51316.9,-0.431957,-0.370467,0.165536,-0.363995,0.834925,4.0691e10,3.6717e10,1.9831e10,3.6343e10,1.2597e10,0.054054,-0.113636,-0.04878,-0.025,0.054054,74.0,88.0,82.0,80.0,74.0,41200.1335,1.590769,"""SobreComprado""",null
2024-03-20 00:00:00,61974.1,68140.0,60800.0,67876.4,637362.465,2024-03-20 23:59:59.999,4.0691e10,8081289,322745.423,2.0619e10,"""BTCUSDT""",74,"""Greed""",0.095238,-0.071809,0.026438,0.086233,0.30839,61974.1,73127.6,66128.1,62487.9,51877.8,-0.096203,0.937979,0.153288,-0.334257,1.572958,4.5022e10,2.0997e10,3.5283e10,6.1121e10,1.5815e10,-0.063291,-0.08642,-0.013333,-0.097561,-0.051282,79.0,81.0,75.0,82.0,78.0,41002.2445,1.655431,"""SobreComprado""",null
2024-03-19 00:00:00,67660.0,68178.6,61582.8,61974.1,700466.795,2024-03-19 23:59:59.999,4.5022e10,8883840,341387.21,2.1945e10,"""BTCUSDT""",79,"""Extreme Greed""",-0.084036,-0.133268,-0.027818,0.085846,0.185032,67660.0,71503.2,63747.4,57074.5,52297.4,1.091539,0.336772,-0.345466,0.555554,1.227832,2.1526e10,3.3680e10,6.8785e10,2.8943e10,2.0209e10,0.025974,-0.024691,-0.122222,0.0,0.097222,77.0,81.0,90.0,79.0,72.0,40792.1415,1.519266,"""SobreComprado""",null
2024-03-18 00:00:00,68447.9,69044.4,66601.6,67660.0,317885.916,2024-03-18 23:59:59.999,2.1526e10,4929189,157518.056,1.0667e10,"""BTCUSDT""",77,"""Extreme Greed""",-0.011511,-0.062281,-0.009318,0.241204,0.30583,68447.9,72153.8,68296.4,54511.6,51813.8,0.023665,-0.36159,-0.482636,0.031949,1.101518,2.1028e10,3.3718e10,4.1607e10,2.0860e10,1.0243e10,-0.025316,-0.060976,-0.060976,0.069444,0.026667,79.0,82.0,82.0,72.0,75.0,40611.249,1.666041,"""SobreComprado""",null
2024-03-17 00:00:00,65351.0,68972.4,64559.8,68447.9,314890.309,2024-03-17 23:59:59.999,2.1028e10,4441202,157746.931,1.0539e10,"""BTCUSDT""",79,"""Extreme Greed""",0.047389,-0.008027,0.083405,0.322284,0.311868,65351.0,69001.8,63178.5,51764.9,52175.9,-0.067137,0.299217,0.684884,2.330066,1.434774,2.2542e10,1.6185e10,1.2481e10,6.3147e9,8.6367e9,-0.024691,0.0,-0.048193,0.067568,0.097222,81.0,79.0,83.0,74.0,72.0,40402.589,1.694146,"""SobreComprado""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-06-30 00:00:00,60982.5,63058.4,60705.0,62766.0,147777.915,2024-06-30 23:59:59.999,9.1318e9,1720176,76847.014,4.7488e9,"""BTCUSDT""",47,"""Neutral""",0.029248,-0.006787,-0.058325,-0.099058,-0.074328,60982.4,63194.9,66653.6,69667.1,67805.9,0.881112,0.980869,0.708637,0.930719,0.315741,4.8545e9,4.6100e9,5.3445e9,4.7297e9,6.9404e9,0.566667,-0.145455,-0.338028,-0.373333,-0.356164,30.0,55.0,71.0,75.0,73.0,58123.2925,1.079877,null,"""right_only"""
2024-06-29 00:00:00,60429.8,61232.0,60400.1,60982.4,79680.289,2024-06-29 23:59:59.999,4.8545e9,1008124,39587.752,2.4118e9,"""BTCUSDT""",30,"""Fear""",0.009144,-0.05071,-0.079006,-0.120613,-0.100544,60429.8,64240.0,66213.7,69346.5,67799.2,-0.594215,0.402093,0.012537,0.063622,0.406815,1.1963e10,3.4623e9,4.7944e9,4.5641e9,3.4507e9,-0.361702,-0.433962,-0.594595,-0.583333,-0.583333,47.0,53.0,74.0,72.0,72.0,58023.8375,1.050989,null,"""right_only"""
2024-06-28 00:00:00,61682.1,62204.0,60038.0,60429.8,195508.125,2024-06-28 23:59:

In [27]:
#Exportar em formato Parquet
dataset_concat_pl.write_parquet('BTC_OFICIAL.parquet')

#Exportar em formato Excel
dataset_concat_pl.write_excel('BTC_OFICIAL.xlsx')